In [ ]:
#@markdown #Connect the Google Drive
#@markdown - Make sure that you are connected to a GPU runtime like `[T4, L4, A100, V100]`
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install huggingface_hub
!pip -q uninstall realesrgan

Proceed (Y/n)? y


In [ ]:
#@markdown #Dependencies for Inferencing

!pip install -q transformers diffusers accelerate

!pip install -q xformers

!pip install -q huggingface_hub

!pip install -q pandas

!pip install -q torchvision

!pip install -q git+https://github.com/sberbank-ai/Real-ESRGAN.git

!pip install -q peft


  Preparing metadata (setup.py) ... done


In [ ]:
#@markdown #Hugging Face Notebook Login
#@markdown - Please visit this [HuggingFace](https://huggingface.co/) website and Login/Signup your account
#@markdown - Go to `Settings`
#@markdown - In `Auth Token` section create an auth token with `write` access
#@markdown - Copy the auth token and paste it here
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
import random
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import pipeline
from PIL import Image
import numpy as np
from diffusers.utils import load_image
from diffusers import StableDiffusionPipeline,StableDiffusionImg2ImgPipeline
import torch
from diffusers import ControlNetModel, UniPCMultistepScheduler
import pandas as pd

# Initialize an empty list to store data
data_list = []
!mkdir output
!mkdir sr_output
!mkdir rejected_images
#@markdown #Data Generation
#@markdown ----
#@markdown
#@markdown - Enter the same `Username` for your HuggingFace Account
username = "mihirkothari0702" #@param{type:"string"}
#@markdown
#@markdown - Enter the same `Session name` used before
Session_Name = "testtt" #@param {type: "string"}

IMAGE_GENERATION_TYPE = "text_to_image" #@param ["text_to_image","image_to_image"]

#@markdown ----
import os
import random
from PIL import Image

def get_random_image_from_folder(folder_path):
    # Get a list of all files in the folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Filter only image files (assuming common image file extensions)
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
    images = [f for f in files if any(f.lower().endswith(ext) for ext in image_extensions)]

    # Check if there are no images in the folder
    if not images:
        raise ValueError("No image files found in the folder")

    # Choose a random image file
    random_image_file = random.choice(images)

    # Open and return the image
    image_path = os.path.join(folder_path, random_image_file)
    image = Image.open(image_path)
    return image

def get_random_prompt_from_folder(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if not files:
        raise ValueError("No files found in the folder")
    random_file = random.choice(files)
    with open(os.path.join(folder_path, random_file), 'r') as file:
        prompt = file.read().strip()
    return prompt

# Function to extract features from an image
def extract_features(image, model, transform, device):
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(image).cpu().numpy().flatten()
    return features

# Preprocess images and compute their feature vectors
def preprocess_acceptable_images(folder_path, model, transform, device):
    acceptable_features = []
    for file_name in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file_name)):
            image = Image.open(os.path.join(folder_path, file_name)).convert('RGB')
            features = extract_features(image, model, transform, device)
            acceptable_features.append(features)
    return np.array(acceptable_features)

# Folder paths
folder_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/captions'
acceptable_images_folder = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/instance_images'

rejected_images_folder = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/rejected_images'
if IMAGE_GENERATION_TYPE == 'text_to_image':
  pipe = StableDiffusionPipeline.from_single_file(f"/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/{Session_Name}.ckpt")
elif IMAGE_GENERATION_TYPE == 'image_to_image':
    pipe = StableDiffusionImg2ImgPipeline.from_single_file(f"/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/{Session_Name}.ckpt")
else:
  print("Please Choose Valid Option")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
# Remove if you do not have xformers installed
# see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
# for installation instructions
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()
# Number of iterations for the loop
number_of_images = 40 #@param {type:"slider", min:10, max:100, step:1}

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load RealESRGAN model
#from RealESRGAN import RealESRGAN
#esrgan_model = RealESRGAN(device, scale=4)
#esrgan_model.load_weights('weights/RealESRGAN_x4.pth', download=True)

# Load pre-trained ResNet model for feature extraction
resnet = models.resnet50(pretrained=True).to(device)
resnet.eval()
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the final classification layer

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Preprocess acceptable images
acceptable_features = preprocess_acceptable_images(acceptable_images_folder, feature_extractor, transform, device)

# Loop to run the inference with random prompts and upscale the generated images
for i in range(number_of_images):
    prompt = get_random_prompt_from_folder(folder_path)
    print(f"Iteration {i+1}: Using prompt - {prompt}")
    if IMAGE_GENERATION_TYPE == "text_to_image":
    # Call your stable diffusion pipeline inference here
      image = pipe(prompt, num_inference_steps=40).images[0]
    elif IMAGE_GENERATION_TYPE == "image_to_image":
      input_image = get_random_image_from_folder(acceptable_images_folder)
      image = pipe(prompt, input_image, num_inference_steps=40).images[0]
    # Save the generated image

    output_image_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/temp_output/output_{i+1}.png'
    image.save(output_image_path)

    # Load the generated image for upscaling
    image = Image.open(output_image_path).convert('RGB')

    # Predict the upscaled image using RealESRGAN
    #sr_image = esrgan_model.predict(image)

    # Save the upscaled image temporarily
    #sr_output_image_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/sr_output/sr_output_{i+1}.png'
    #sr_image.save(sr_output_image_path)

    # Extract features of the upscaled image
    generated_features = extract_features(image, feature_extractor, transform, device)

    # Compute similarity with acceptable images
    similarities = cosine_similarity([generated_features], acceptable_features)
    max_similarity = similarities.max()

    # Check if the image meets the acceptance threshold
    similarity_threshold = 0.9  # Adjust this threshold based on your needs
    if max_similarity > similarity_threshold:
        final_output_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/Outputs/final_output_{i+1}.png'
        image.save(final_output_path)
        print(f"Saved upscaled image at: {final_output_path}")
        # Create a dictionary representing a row of data
        row_data = {
            "No.": i+1,
            "Prompt": prompt,
            "Image": final_output_path,
            "Similarity_Score": max_similarity
        }
        # Append the row data to the list
        data_list.append(row_data)

    else:
        rejected_output_path = os.path.join(rejected_images_folder, f'rejected_output_{i+1}.png')
        image.save(rejected_output_path)
        print(f"Saved rejected image at: {rejected_output_path}")
        row_data = {
            "No.": i+1,
            "Prompt": prompt,
            "Image": rejected_output_path,
            "Similarity_Score": max_similarity
        }
    df = pd.DataFrame(data_list)
    csv_file_path = '/content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/Outputs/Data_Generation.csv'
    df.to_csv(csv_file_path, index=False)


mkdir: cannot create directory ‘output’: File exists
mkdir: cannot create directory ‘sr_output’: File exists
mkdir: cannot create directory ‘rejected_images’: File exists


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Iteration 1: Using prompt - A yellow pill, round in shape, securely housed in an aluminum blister pack with 4 circular slots organized into 4 rows and 2 columns, lying horizontally.


  0%|          | 0/40 [00:00<?, ?it/s]

Saved rejected image at: /content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/rejected_images/rejected_output_1.png


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Outputs'

In [ ]:
for i in range(number_of_images):
    prompt = get_random_prompt_from_folder(folder_path)
    print(f"Iteration {i+1}: Using prompt - {prompt}")
    if IMAGE_GENERATION_TYPE == "text_to_image":
    # Call your stable diffusion pipeline inference here
      image = pipe(prompt, num_inference_steps=40).images[0]
    elif IMAGE_GENERATION_TYPE == "image_to_image":
      input_image = get_random_image_from_folder(acceptable_images_folder)
      image = pipe(prompt, input_image, num_inference_steps=40).images[0]
    # Save the generated image

    output_image_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/temp_output/output_{i+1}.png'
    image.save(output_image_path)

    # Load the generated image for upscaling
    image = Image.open(output_image_path).convert('RGB')

    # Predict the upscaled image using RealESRGAN
    #sr_image = esrgan_model.predict(image)

    # Save the upscaled image temporarily
    #sr_output_image_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/sr_output/sr_output_{i+1}.png'
    #sr_image.save(sr_output_image_path)

    # Extract features of the upscaled image
    generated_features = extract_features(image, feature_extractor, transform, device)

    # Compute similarity with acceptable images
    similarities = cosine_similarity([generated_features], acceptable_features)
    max_similarity = similarities.max()

    # Check if the image meets the acceptance threshold
    similarity_threshold = 0.9  # Adjust this threshold based on your needs
    if max_similarity > similarity_threshold:
        final_output_path = f'/content/drive/MyDrive/Fast-Dreambooth/Sessions/{Session_Name}/Outputs/final_output_{i+1}.png'
        image.save(final_output_path)
        print(f"Saved upscaled image at: {final_output_path}")
        # Create a dictionary representing a row of data
        row_data = {
            "No.": i+1,
            "Prompt": prompt,
            "Image": final_output_path,
            "Similarity_Score": max_similarity
        }
        # Append the row data to the list
        data_list.append(row_data)

    else:
        rejected_output_path = os.path.join(rejected_images_folder, f'rejected_output_{i+1}.png')
        image.save(rejected_output_path)
        print(f"Saved rejected image at: {rejected_output_path}")
        row_data = {
            "No.": i+1,
            "Prompt": prompt,
            "Image": rejected_output_path,
            "Similarity_Score": max_similarity
        }
    df = pd.DataFrame(data_list)
    csv_file_path = '/content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/Outputs/Data_Generation.csv'
    df.to_csv(csv_file_path, index=False)


Iteration 1: Using prompt - A horizontally positioned blister pack made of aluminum, containing a white round pill with a deep incision on one side that fits perfectly in one of its 8 circular compartments arranged in 4 rows and 2 columns.


  0%|          | 0/40 [00:00<?, ?it/s]

Saved rejected image at: /content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/rejected_images/rejected_output_1.png
Iteration 2: Using prompt - An aluminum blister pack with 4 round cavities, organized in 2 rows by 2 columns, holding a green pill and positioned horizontally.


  0%|          | 0/40 [00:00<?, ?it/s]

Saved rejected image at: /content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/rejected_images/rejected_output_2.png
Iteration 3: Using prompt - An oval-shaped orange pill wrapped in aluminum foil, featuring a slight cut in the center on one side, all pills are securely positioned horizontally oriented in a blister pack with 6 oval slots arranged in 3 rows and 2 columns.


...

Saved rejected image at: /content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/rejected_images/rejected_output_5.png
Iteration 6: Using prompt - An aluminum blister pack holding a perfectly fitting round yellow pill in one of its 10 circular cavities, arranged in a vertical grid of 5 rows and 2 columns.


  0%|          | 0/40 [00:00<?, ?it/s]

Saved rejected image at: /content/drive/MyDrive/Fast-Dreambooth/Sessions/testtt/rejected_images/rejected_output_6.png
Iteration 7: Using prompt - An oval-shaped light green pill wrapped in aluminum foil, all pills are securely positioned  vertically oriented in a blister pack featuring 14 oval slots arranged in 7 rows and 2 columns.


  0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from diffusers.utils import load_image
from diffusers import StableDiffusionPipeline,StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
from diffusers import ControlNetModel, UniPCMultistepScheduler
#@markdown #Simple Model Inferencing(Model Testing)
#@markdown - Enter the same session name
Session_Name = "testtt" #@param {type: "string"}
#@markdown - Enter the Prompt for images generation
#@markdown - This will generate images from model but will not `upscale` them
#@markdown - Image will be stored at this `/content/test_output/test_output.png` address
#@markdown - Choose the Image Generation Type
IMAGE_GENERATION = "text_to_image" #@param ["text_to_image","image_to_image"]
if IMAGE_GENERATION == 'text_to_image':
  pipe = StableDiffusionPipeline.from_single_file(f"https://huggingface.co/riddhiparmar/PVC2o_Orange2o_Bilayer2o/blob/main/PVC20_Orange20_Bilayer20.ckpt")
elif IMAGE_GENERATION == 'image_to_image':
    pipe = StableDiffusionImg2ImgPipeline.from_single_file(f"https://huggingface.co/riddhiparmar/PVC2o_Orange2o_Bilayer2o/blob/main/PVC20_Orange20_Bilayer20.ckpt")

else:
  print("Please Choose Valid Option")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
# Remove if you do not have xformers installed
# see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
# for installation instructions
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of the model checkpoint were not used when initializing CLIPTextModel: 
 ['text_model.embeddings.position_ids']
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
The following part of your i

  0%|          | 0/32 [00:00<?, ?it/s]

mkdir: cannot create directory ‘test_output’: File exists


In [ ]:
Prompt = "A striking macro photograph of a round, dark blue pill perfectly positioned within a round-shaped hole. The pill casts a distinct shadow that highlights its three-dimensional form and depth. The background is stark white, emphasizing the pill's contrasting color and creating a striking visual impact. The overall composition of the image is simple yet effective, drawing the viewer's attention to the pill and its placement within the hole." #@param{type:"string"}
inference_steps = 30 #@param{type:"number"}
#@markdown - This is only for image_to_image
image_path = "/content/1.png" #@param {type:'string'}

if IMAGE_GENERATION == "text_to_image":
    # Call your stable diffusion pipeline inference here
  image = pipe(Prompt, num_inference_steps=40).images[0]
elif IMAGE_GENERATION == "image_to_image":
  input_image = Image.open(image_path).convert('RGB')
  image = pipe(Prompt, input_image, num_inference_steps=40).images[0]
!mkdir test_output
image.save(f'/content/test_output/test_output.png')